In [15]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import pandas as pd
import re
import numpy as np 
import math
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler
from deap import algorithms, base, creator, tools
from nltk import word_tokenize
import re, string
import random
import statistics


[nltk_data] Downloading package punkt to /home/spetz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# class constants
num_words = 10  # number of words to print
n_gen = 150  # Increase to possibly improve best individual
pop_size = 250
prob_xover = 0.5
prob_mut = 0.4
tournament_size = 3
num_iterations = 10
dict_freq = {}  # frequency map for words in list
dict_paired_freq = {}

path = '/home/spetz/Downloads/DeliciousMIL/Data/train-data.dat'
clean_files = []
df = pd.DataFrame()

file = open(path).readlines()
len(file)

#clear data
clear_file=[]
for i in range(len(file)):
    x=re.sub('<.*?>','',file[i])
    clear_file.append(x)

words = ''
for line in clear_file:
    words += line

regex = re.compile('[,.;\"\'!?_-]')
words = regex.sub('', words)
tokenized_words = word_tokenize(words) # list of all words in new dictionary
WORD_LIST = list(set(tokenized_words)) # create a set out of words so there are no repeats in word list and make list again
dictionary_size = len(WORD_LIST)

In [4]:
for word in tokenized_words:
    if word not in dict_freq:
        dict_freq[word] = 1
    else:
        dict_freq[word] += 1
dict_freq

{'6705': 554,
 '5997': 378,
 '8310': 46,
 '3606': 301,
 '674': 6376,
 '8058': 407,
 '5044': 120,
 '4836': 1018,
 '4312': 3478,
 '5154': 4695,
 '4225': 552,
 '1827': 311,
 '1037': 742,
 '8482': 718,
 '483': 834,
 '3567': 35,
 '6172': 726,
 '2892': 2270,
 '1362': 1254,
 '787': 622,
 '399': 1353,
 '777': 2175,
 '1332': 1720,
 '318': 135,
 '769': 89,
 '4621': 3250,
 '3199': 1022,
 '1480': 233,
 '6213': 285,
 '971': 1100,
 '6890': 3897,
 '5909': 2003,
 '15': 26,
 '3445': 46,
 '2475': 132,
 '324': 45,
 '4138': 72,
 '3404': 96,
 '6176': 8,
 '65': 2722,
 '2926': 324,
 '1375': 613,
 '7705': 10,
 '709': 9,
 '1323': 162,
 '1652': 6,
 '5735': 567,
 '7439': 2506,
 '4396': 222,
 '4382': 1151,
 '1798': 16,
 '1870': 59,
 '1845': 67,
 '4410': 128,
 '7758': 6,
 '4278': 10,
 '7775': 25,
 '2898': 44,
 '1923': 111,
 '8193': 45,
 '2930': 617,
 '485': 24,
 '5951': 139,
 '6123': 67,
 '2480': 90,
 '3158': 1616,
 '2193': 31,
 '2438': 8,
 '7485': 22,
 '3412': 577,
 '5304': 78,
 '5644': 112,
 '3734': 9,
 '7459': 

In [5]:
#TF = (Frequency of the word in the sentence) / (Total number of words in the sentence)
def computeTF(wordDict,bow):
    tfDict={}
    bowCount=len(bow)
    for word,count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

tf=computeTF(dict_freq,tokenized_words)
tf

{'6705': 0.000550918163791749,
 '5997': 0.0003758972308903991,
 '8310': 4.57441074628528e-05,
 '3606': 0.00029932557274605856,
 '674': 0.006340531069198902,
 '8058': 0.0004047359073343715,
 '5044': 0.00011933245425092036,
 '4836': 0.0010123369868953076,
 '4312': 0.003458652299039175,
 '5154': 0.004668882272567259,
 '4225': 0.0005489292895542336,
 '1827': 0.00030926994393363523,
 '1037': 0.0007378723421181909,
 '8482': 0.0007140058512680068,
 '483': 0.0008293605570438965,
 '3567': 3.4805299156518435e-05,
 '6172': 0.0007219613482180681,
 '2892': 0.00225737225957991,
 '1362': 0.0012470241469221177,
 '787': 0.0006185398878672705,
 '399': 0.001345473421679127,
 '777': 0.002162900733297931,
 '1332': 0.0017104318442631918,
 '318': 0.0001342490110322854,
 '769': 8.850490356943259e-05,
 '4621': 0.003231920635962426,
 '3199': 0.0010163147353703383,
 '1480': 0.00023170384867053702,
 '6213': 0.0002834145788459358,
 '971': 0.0010938808306334366,
 '6890': 0.0038753214517986386,
 '5909': 0.0019918575

In [6]:
#IDF: log((Total number of sentences (documents))/(Number of sentences (documents) containing the word))
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(dict_freq,0)
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / (float(val) + 1))
        
    return idfDict

idf=computeIDF(dict_freq)
idf


{'6705': 3.9304395947667,
 '5997': 3.9304395947667,
 '8310': 3.9304395947667,
 '3606': 3.9304395947667,
 '674': 3.9304395947667,
 '8058': 3.9304395947667,
 '5044': 3.9304395947667,
 '4836': 3.9304395947667,
 '4312': 3.9304395947667,
 '5154': 3.9304395947667,
 '4225': 3.9304395947667,
 '1827': 3.9304395947667,
 '1037': 3.9304395947667,
 '8482': 3.9304395947667,
 '483': 3.9304395947667,
 '3567': 3.9304395947667,
 '6172': 3.9304395947667,
 '2892': 3.9304395947667,
 '1362': 3.9304395947667,
 '787': 3.9304395947667,
 '399': 3.9304395947667,
 '777': 3.9304395947667,
 '1332': 3.9304395947667,
 '318': 3.9304395947667,
 '769': 3.9304395947667,
 '4621': 3.9304395947667,
 '3199': 3.9304395947667,
 '1480': 3.9304395947667,
 '6213': 3.9304395947667,
 '971': 3.9304395947667,
 '6890': 3.9304395947667,
 '5909': 3.9304395947667,
 '15': 3.9304395947667,
 '3445': 3.9304395947667,
 '2475': 3.9304395947667,
 '324': 3.9304395947667,
 '4138': 3.9304395947667,
 '3404': 3.9304395947667,
 '6176': 3.930439594766

In [7]:
#FINAL TF-IDF
def computeDFIDF(tfbow,idfs):
    tfidf={}
    for word,val in tfbow.items():
        tfidf[word] = val*idfs[word]

    return tfidf

In [8]:
Tf_idf=computeDFIDF(tf,idf)
new_list = list(Tf_idf.values())
len(new_list)
tf_idf_Xtrain = np.asarray(new_list)


In [9]:
# create a frequency map of paired words in the text
for i in range(len(tokenized_words) - 1):
    if (tokenized_words[i+1],tokenized_words[i]) not in dict_paired_freq:
        dict_paired_freq[(tokenized_words[i+1],tokenized_words[i])] = 1
    else:
        dict_paired_freq[(tokenized_words[i+1],tokenized_words[i])] += 1

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [10]:
def getFitness(individual):
    ind_sentence = ''
    fitness = 0
    for word_index in individual:
        ind_sentence += tokenized_words[word_index] + ' '
        if dict_freq[tokenized_words[word_index]] > 1000:
            fitness -= 2
    tokenized_sentence = word_tokenize(ind_sentence)
    for i in range(len(tokenized_sentence) - 1):
        if (tokenized_sentence[i + 1], tokenized_sentence[i]) in dict_paired_freq:
            fitness += dict_paired_freq[(tokenized_sentence[i+1], tokenized_sentence[i])]
    return fitness,


In [16]:
toolbox = base.Toolbox()
# '6randomly placed words in a list ranging from 0-dictionary_size
toolbox.register("indices", random.sample, range(dictionary_size), num_words)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", getFitness)
toolbox.register("mate", tools.cxUniform, indpb=0.05)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=dictionary_size, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=tournament_size)

top_sentences = []

# Repeat algorithm ten times over 150 generations
for num_sentence in range(num_iterations):
    pop = toolbox.population(n=pop_size)
    bestfitlist = []
    bestfitval = 0;
    # Let's collect the fitness values from the simulation using
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(0, n_gen):
        # Start creating the children (or offspring)

        # First, Apply selection:
        offspring = toolbox.select(pop, pop_size)
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply variations (xover and mutation), Ex: algorithms.varAnd(?, ?, ?, ?)
        offspring = algorithms.varAnd(offspring, toolbox, prob_xover, prob_mut)

        for fit in fitnesses:
            if bestfitval < fit[0]:
                bestfitval = fit[0]
        bestfitlist.append(bestfitval)

        # Let's evaluate the fitness of each individual.

        fitnesses = list(map(toolbox.evaluate, offspring))

    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit
    # One way of implementing elitism is to combine parents and children to give them equal chance to compete:
    # For example: pop[:] = pop + offspring
    # Otherwise you can select the parents of the generation from the offspring population only: pop[:] = offspring
    pop[:] = pop + offspring
    print("-- End of evolution --  Number of generations: " + str(n_gen))
    print("Best fitness values list: %s" % (bestfitlist))
    print("average fitness: is", statistics.mean(bestfitlist))

    best_ind = tools.selBest(pop, 1)[0]
    best_ind_sentence = ''
    for i in best_ind:
        if i > dictionary_size:
            i = dictionary_size - 1
        best_ind_sentence += tokenized_words[i] + ' '

    display_sentence = "Sentence %s: \"%s\", best fitness: %s" % (num_sentence + 1, best_ind_sentence, bestfitlist[len(bestfitlist) - 1])
    print(display_sentence)
    top_sentences.append(display_sentence)

print("-- End of the %d evolutions --\n" % num_iterations)
best_ind = tools.selBest(pop, 1)[0]
print("The Top %d Sentences Generated:" % num_words)
for sentence in top_sentences:
    print(sentence)

-- End of evolution --  Number of generations: 150
Best fitness values list: [53, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 308, 308, 308, 308, 308, 308, 308, 308, 308, 308, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 339, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541, 541]
average fitness: is 428.6933333333333
Sentence 1: "4507 94 7412 4096 6 3967 7654 470 2287 1037 ", best fitness: 541
-- End of evolution --  Number of generations: 150
Best f